# Predicting outcome of chess games based on my performance this year

In [10]:
import numpy as np
import pandas as pd

from keras.models import Sequential
from keras.layers import Dense

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score

from time import time

from keras.wrappers.scikit_learn import KerasClassifier

from dummies_bins_test_train_cv import get_Xy_train_test
from different_xy import diff_bins, xy_custom

## If new data is added, uncomment and run the next line

In [11]:
from clean_chess_game_log import main_cleanup
_,_,_ = main_cleanup('../data/dest.pgn')

In [12]:
df = pd.read_csv('../data/use_for_predictions.csv')

In [14]:
df = df.loc[df['result'] != 0.5].copy()
df.reset_index(inplace=True)
df.drop(columns=['index'], inplace=True)

In [15]:
df, y = clean_df_y(df)

NameError: name 'clean_df_y' is not defined

## Creating the classifier:
__The _Fun_ Part!!__

### Ran the classifier with new data every time I played on Chess.com.

Which is how I ended up with the following steps:

- __Step 1__
    - Add the sequential layer
- __Step 2__
    - Add the __Dense__ layer with following parameters:
        - Units = 128
        - Initializer = Uniform
        - Activation = Softmax (Since the outcome is either 1 or 0, Softmax works wonders)
        - Input Dim = the second part of the X Shape (# of columns)
- __Step 3__
    - Add the second __Dense__ layer with following parameters:
        - Units = 256 (Double it up!!)
        - Initializer = Normal (Throw a curve ball!)
        - Activation = ReLu (Prevents Vanashing Gradiant problem)
- __Step 4__
    - Add the third __Dense__ layer with following parameters:
        - Units = 64 (Make it easier for the compiling)
        - Initializer = Uniform
        - Activation = Softmax
- __Step 5__
    - Add the final __Dense__ layer with following parameters:
        - Units = 1 (Just need 1 column for output)
        - Initializer = Normal (#MuscleConfusion. I kid!)
        - Activation = Sigmoid
- __Step 6__
    - Only think left is to __COMPILE__ it. From all the parameters I've tested. These work the best:
        - Optimizer = Nadam (Nesterov Adam optimizer - Nadam is Adam RMSprop with Nesterov momentum)
        - loss = Mean Absolute Error (Didn't get great results with binary_crossentropy, even though it was my first choice. Mean Squared Error, made error very small, since the loss is between 0 and 1.)
        - metrics = Binary Accuracy (Only have 0 or 1 to predict)

In [16]:
def _classifier():

    classifier = Sequential()
    
    classifier.add(Dense(units=64, activation='softmax', input_dim=X.shape[1]))
    
    classifier.add(Dense(units=128, activation='relu'))
    
    classifier.add(Dense(units=32, activation='softmax'))
    
    classifier.add(Dense(units=1, activation='sigmoid'))
    
    classifier.compile(
        optimizer='nadam', loss='binary_crossentropy', metrics=['accuracy'])

    return classifier

## Load the data to run predictions

## Create Train and Test sets, along with clean the data
### Also, see the shape to know the data is in right shape

In [7]:
all_cols_try = [
    ['diff_bin'], ['color'], ['game_time'], ['time_bin'], ['weekday'],
    ['diff_bin', 'color'], ['diff_bin', 'time_bin'], ['diff_bin', 'game_time'],
    ['diff_bin', 'weekday'], ['diff_bin', 'color', 'time_bin'],
    ['diff_bin', 'color', 'game_time'], ['diff_bin', 'color', 'weekday'],
    ['diff_bin', 'time_bin', 'game_time'], ['diff_bin', 'time_bin', 'weekday'],
    ['diff_bin', 'game_time', 'weekday'],
    ['diff_bin', 'color', 'time_bin', 'game_time'],
    ['diff_bin', 'color', 'time_bin', 'weekday'],
    ['diff_bin', 'color', 'game_time', 'weekday'],
    ['diff_bin', 'time_bin', 'game_time', 'weekday'],
    ['diff_bin', 'color', 'game_time', 'time_bin', 'weekday']]

In [8]:
results = {}

In [7]:
for clm in all_cols_try:
    st = time()
    X_train, X_test, y_train, y_test, X = xy_custom(df, y, 100, clm)
    std_sclr = StandardScaler()
    X_train = std_sclr.fit_transform(X_train)
    X_test = std_sclr.fit_transform(X_test)
    for bs in [6, 12, 24, 36, 48, 96]:
        for ep in [48, 96, 192]:
            classifier = _classifier()
            classifier.fit(X_train, y_train, batch_size=bs, epochs=ep,
                          class_weight='balanced', shuffle=False, verbose=2)
            y_pred = classifier.predict(X_test)
            y_pred = (y_pred > 0.5)
            cm = confusion_matrix(y_test, y_pred)
            results[(f'cols:{clm}, bs:{bs}, ep:{ep} (cm)')] = list(cm)
            results[(f'cols:{clm}, bs:{bs}, ep:{ep} (accuracy)')
                    ] = (f'{((cm[0][0]+cm[1][1])/cm.sum()*100).round(1)}%')
        print(clm)
    print(results)
    print(time() - st)

y Shape: (2117,)
X Shape: (2117, 1)
X_train Shape: (2011, 1)
X_test Shape: (106, 1)
y_train Shape: (2011,)
y_test Shape: (106,)
Epoch 1/96
2011/2011 [==============================] - 1s 447us/step - loss: 0.4838 - acc: 0.6101
Epoch 2/96
2011/2011 [==============================] - 0s 185us/step - loss: 0.4212 - acc: 0.6937
Epoch 3/96
2011/2011 [==============================] - 0s 172us/step - loss: 0.3932 - acc: 0.6902
Epoch 4/96
2011/2011 [==============================] - 0s 173us/step - loss: 0.3757 - acc: 0.6917
Epoch 5/96
2011/2011 [==============================] - 0s 179us/step - loss: 0.3646 - acc: 0.6912
Epoch 6/96
2011/2011 [==============================] - 0s 190us/step - loss: 0.3548 - acc: 0.6912
Epoch 7/96
2011/2011 [==============================] - 0s 173us/step - loss: 0.3453 - acc: 0.6972
Epoch 8/96
2011/2011 [==============================] - 0s 175us/step - loss: 0.3384 - acc: 0.7001
Epoch 9/96
2011/2011 [==============================] - 0s 181us/step - loss: 0.

Epoch 65/196
2011/2011 [==============================] - 0s 204us/step - loss: 0.3001 - acc: 0.7001
Epoch 66/196
2011/2011 [==============================] - 0s 172us/step - loss: 0.3001 - acc: 0.7001
Epoch 67/196
2011/2011 [==============================] - 0s 178us/step - loss: 0.3001 - acc: 0.7001
Epoch 68/196
2011/2011 [==============================] - 0s 185us/step - loss: 0.3000 - acc: 0.7001
Epoch 69/196
2011/2011 [==============================] - 0s 174us/step - loss: 0.3000 - acc: 0.7001
Epoch 70/196
2011/2011 [==============================] - 0s 200us/step - loss: 0.3000 - acc: 0.7001
Epoch 71/196
2011/2011 [==============================] - 0s 172us/step - loss: 0.3000 - acc: 0.7001
Epoch 72/196
2011/2011 [==============================] - 0s 173us/step - loss: 0.3000 - acc: 0.7001
Epoch 73/196
2011/2011 [==============================] - 0s 154us/step - loss: 0.3000 - acc: 0.7001
Epoch 74/196
2011/2011 [==============================] - 0s 161us/step - loss: 0.3000 - ac

2011/2011 [==============================] - 0s 95us/step - loss: 0.3176 - acc: 0.6967
Epoch 28/96
2011/2011 [==============================] - 0s 82us/step - loss: 0.3167 - acc: 0.6967
Epoch 29/96
2011/2011 [==============================] - 0s 78us/step - loss: 0.3158 - acc: 0.6967
Epoch 30/96
2011/2011 [==============================] - 0s 90us/step - loss: 0.3150 - acc: 0.6967
Epoch 31/96
2011/2011 [==============================] - 0s 81us/step - loss: 0.3140 - acc: 0.6967
Epoch 32/96
2011/2011 [==============================] - 0s 83us/step - loss: 0.3132 - acc: 0.6977
Epoch 33/96
2011/2011 [==============================] - 0s 85us/step - loss: 0.3124 - acc: 0.6977
Epoch 34/96
2011/2011 [==============================] - 0s 78us/step - loss: 0.3117 - acc: 0.6977
Epoch 35/96
2011/2011 [==============================] - 0s 77us/step - loss: 0.3111 - acc: 0.6977
Epoch 36/96
2011/2011 [==============================] - 0s 77us/step - loss: 0.3106 - acc: 0.6987
Epoch 37/96
2011/2011 

KeyboardInterrupt: 

In [ ]:
# X_train, X_test, y_train, y_test, X, y = 

In [4]:
# X_train, X_test, y_train, y_test = get_Xy_train_test(
#     df, .98, .99)

y Shape: (2117,)
X Shape: (2117, 1)
X_train Shape: (2093, 1)
X_test Shape: (24, 1)
y_train Shape: (2093,)
y_test Shape: (24,)


## Scale the data for easier computations

In [6]:
# std_sclr = StandardScaler()
# X_train = std_sclr.fit_transform(X_train)
# X_test = std_sclr.fit_transform(X_test)

### See the scale of each value if interested

In [7]:
# std_sclr.scale_

array([10.59603896])

## Fitting the training data

### I wanted to test different batch sizes to get the results on the real games. 

I started testing fitting classifier __For Loop__ with the following parameters:

1. Batch Size:
    1. 20
    2. 16
    3. 12
    4. 8
    5. 6
2. Epochs:
    1. 50
    2. 100
    3. 200

In [9]:
results = {}

In [10]:
for bs in [20, 16, 12, 8, 6]:
    for ep in [50, 100, 200]:
        classifier = _classifier()
        classifier.fit(X_train, y_train, batch_size=bs, epochs=ep,
                       class_weight='balanced', shuffle=False)
        y_pred = classifier.predict(X_test)
        y_pred = (y_pred > 0.5)
        cm = confusion_matrix(y_test, y_pred)
        results[(f'bs:{bs}, ep:{ep} (cm)')] = cm
        results[(f'bs:{bs}, ep:{ep} (accuracy)')
                ] = f'{((cm[0][0]+cm[1][1])/cm.sum()*100).round(1)}%'
        print(results)

Epoch 1/50
2093/2093 [==============================] - 1s 380us/step - loss: 0.6933 - acc: 0.5055
Epoch 2/50
2093/2093 [==============================] - 0s 140us/step - loss: 0.6931 - acc: 0.5055
Epoch 3/50
2093/2093 [==============================] - 0s 134us/step - loss: 0.6917 - acc: 0.5093
Epoch 4/50
2093/2093 [==============================] - 0s 147us/step - loss: 0.6842 - acc: 0.6603
Epoch 5/50
2093/2093 [==============================] - 0s 141us/step - loss: 0.6635 - acc: 0.6971
Epoch 6/50
2093/2093 [==============================] - 0s 144us/step - loss: 0.6371 - acc: 0.6933
Epoch 7/50
2093/2093 [==============================] - 0s 133us/step - loss: 0.6160 - acc: 0.6933
Epoch 8/50
2093/2093 [==============================] - 0s 141us/step - loss: 0.6023 - acc: 0.6980
Epoch 9/50
2093/2093 [==============================] - 0s 141us/step - loss: 0.5940 - acc: 0.6985
Epoch 10/50
2093/2093 [==============================] - 0s 149us/step - loss: 0.5888 - acc: 0.6985
Epoch 11/

2093/2093 [==============================] - 0s 134us/step - loss: 0.5889 - acc: 0.6980
Epoch 11/200
2093/2093 [==============================] - 0s 148us/step - loss: 0.5855 - acc: 0.6980
Epoch 12/200
2093/2093 [==============================] - 0s 134us/step - loss: 0.5829 - acc: 0.6980
Epoch 13/200
2093/2093 [==============================] - 0s 140us/step - loss: 0.5807 - acc: 0.6980
Epoch 14/200
2093/2093 [==============================] - 0s 135us/step - loss: 0.5788 - acc: 0.6980
Epoch 15/200
2093/2093 [==============================] - 0s 133us/step - loss: 0.5771 - acc: 0.6980
Epoch 16/200
2093/2093 [==============================] - 0s 135us/step - loss: 0.5755 - acc: 0.6980
Epoch 17/200
2093/2093 [==============================] - 0s 138us/step - loss: 0.5741 - acc: 0.6980
Epoch 18/200
2093/2093 [==============================] - 0s 140us/step - loss: 0.5728 - acc: 0.7000
Epoch 19/200
2093/2093 [==============================] - 0s 133us/step - loss: 0.5716 - acc: 0.7004
Epo

2093/2093 [==============================] - 0s 172us/step - loss: 0.5588 - acc: 0.6957
Epoch 172/200
2093/2093 [==============================] - 0s 141us/step - loss: 0.5587 - acc: 0.6952
Epoch 173/200
2093/2093 [==============================] - 0s 166us/step - loss: 0.5587 - acc: 0.6952
Epoch 174/200
2093/2093 [==============================] - 0s 136us/step - loss: 0.5586 - acc: 0.6952
Epoch 175/200
2093/2093 [==============================] - 0s 163us/step - loss: 0.5586 - acc: 0.6952
Epoch 176/200
2093/2093 [==============================] - 0s 136us/step - loss: 0.5585 - acc: 0.6952
Epoch 177/200
2093/2093 [==============================] - 0s 168us/step - loss: 0.5585 - acc: 0.6952
Epoch 178/200
2093/2093 [==============================] - 0s 142us/step - loss: 0.5584 - acc: 0.6947
Epoch 179/200
2093/2093 [==============================] - 0s 157us/step - loss: 0.5584 - acc: 0.6942
Epoch 180/200
2093/2093 [==============================] - 0s 155us/step - loss: 0.5583 - acc: 0

KeyboardInterrupt: 

### Here are the best & the worst combinations:

### The best:
1. batch Size = 12, Epochs = 50
    - CM = [__11__ 2][4 __13__]
    - Accuracy = 80%
2. batch Size = 12, Epochs = 100
    - CM = [__11__ 2][4 __13__]
    - Accuracy = 80%
3. batch Size = 20, Epochs = 100
    - CM = [__11__ 2][4 __13__]
    - Accuracy = 80%
4. batch Size = 8, Epochs = 50
    - CM = [10 3][4 __13__]
    - Accuracy = 76.67%

### The worst:
1. batch Size = 8, Epochs = 100
    - CM = [11 2] [__7__ 10]
    - Accuracy = 70%
2. batch Size = 16, Epochs = 50
    - CM = [10 3] [__6__ 11]
    - Accuracy = 70%

#### The rest were at 73.33%

NOTE:
- All batches of 6 were at 73%
- All Epochs of 200 were at 73%


In [16]:
classifier = _classifier()
classifier.fit(X_train, y_train, batch_size=20, epochs=100,
               class_weight='balanced', shuffle=False)
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)
cm = confusion_matrix(y_test, y_pred)

Epoch 1/100
2067/2067 [==============================] - 1s 443us/step - loss: 0.5001 - acc: 0.4954
Epoch 2/100
2067/2067 [==============================] - 0s 165us/step - loss: 0.4967 - acc: 0.5317
Epoch 3/100
2067/2067 [==============================] - 0s 160us/step - loss: 0.4549 - acc: 0.6923
Epoch 4/100
2067/2067 [==============================] - 0s 181us/step - loss: 0.4208 - acc: 0.7025
Epoch 5/100
2067/2067 [==============================] - 0s 169us/step - loss: 0.3986 - acc: 0.7112
Epoch 6/100
2067/2067 [==============================] - 0s 193us/step - loss: 0.3822 - acc: 0.7121
Epoch 7/100
2067/2067 [==============================] - 0s 169us/step - loss: 0.3695 - acc: 0.7097
Epoch 8/100
2067/2067 [==============================] - 0s 183us/step - loss: 0.3589 - acc: 0.7112
Epoch 9/100
2067/2067 [==============================] - 0s 216us/step - loss: 0.3472 - acc: 0.7194
Epoch 10/100
2067/2067 [==============================] - 0s 170us/step - loss: 0.3398 - acc: 0.7218

In [17]:
print(cm)
print(f'accuracy: {((cm[0][0]+cm[1][1])/cm.sum()*100).round(1)}%')

[[10  6]
 [ 6 15]]
accuracy: 67.6%
